In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

import tensorflow as tf
import torch
import onnx
from onnx_tf.backend import prepare




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.











In [2]:
import sys

sys.path.append('../InsightFace-v2/')

In [3]:
import numpy as np
from tensorflow.contrib import graph_editor as ge

## Torch to onnx

In [4]:
checkpoint = torch.load('../InsightFace-v2/pretrained/BEST_checkpoint_r101.tar')

In [5]:
model = checkpoint['model'].module

In [6]:
from PIL import Image

image = Image.open("/media/hpc2_storage/mvasil/img_align_celeba/124142.jpg")
image = image.resize((112, 112))
dummy_input = torch.tensor(np.stack([np.array(image), np.array(image)]), dtype=torch.float32).permute(0, 3, 1, 2).cuda() / 255 - 0.5

In [12]:
torch.onnx.export(model, dummy_input, '../InsightFace-v2/pretrained/embedder.onnx',
                  input_names=['emb_input'], output_names=['emb_output'], dynamic_axes={'emb_input': [0]});

## Fix batch

In [13]:
# model_onnx = onnx.load('../InsightFace-v2/pretrained/embedder.onnx')

# #Fix batch
# model_onnx.graph.input[0].type.tensor_type.shape.dim[0].dim_param = '?'
# onnx.save(model_onnx, 'embedder.onnx')

## Onnx to tf

In [14]:
code = '''
from warnings import filterwarnings
filterwarnings('ignore')
import onnx
from onnx_tf.backend import prepare
model_onnx = onnx.load('../InsightFace-v2/pretrained/embedder.onnx')
tf_rep = prepare(model_onnx)
tf_rep.export_graph('../InsightFace-v2/pretrained/embedder.pb')
'''

In [15]:
!CUDA_VISIBLE_DEVICES=0 python -c "{code}"




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.










2020-04-17 16:26:57.089532: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-17 16:26:57.093889: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3408305000 Hz
2020-04-17 16:26:57.094231: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cc6899be80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-17 16:26:57.094250: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-04-17 16

2020-04-17 16:26:57.230662: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:26:57.231200: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:26:57.231256: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:26:57.231275: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-17 16:26:57.231290: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-04-17 16:26:57.231306: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
2020-04-17 16:26:57.322083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:26:57.322578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:26:57.322621: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:26:57.322636: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Succ

2020-04-17 16:26:57.416391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:26:57.416892: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:26:57.416935: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:26:57.416949: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-17 16:26:57.416963: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-04-17 16:26:57.416975: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

2020-04-17 16:26:57.518071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:26:57.518588: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:26:57.518643: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:26:57.518661: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-17 16:26:57.518674: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-04-17 16:26:57.518687: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

...

2020-04-17 16:27:01.208258: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:27:01.208779: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:27:01.208825: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:27:01.208845: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-17 16:27:01.208865: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-04-17 16:27:01.208877: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

2020-04-17 16:27:01.412876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:27:01.413385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:27:01.413425: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:27:01.413439: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-17 16:27:01.413450: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-04-17 16:27:01.413461: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

2020-04-17 16:27:01.616911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:27:01.617408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:27:01.617448: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:27:01.617461: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-17 16:27:01.617472: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-04-17 16:27:01.617483: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

2020-04-17 16:27:01.816473: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:27:01.819240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:27:01.819291: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-04-17 16:27:01.819307: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-04-17 16:27:01.819318: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0
2020-04-17 16:27:01.819330: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

2020-04-17 16:27:01.880629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:27:01.881100: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1304] Created TensorFlow device (/device:GPU:0 with 8215 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1)
2020-04-17 16:27:02.069593: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-17 16:27:02.070099: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.645
pciBusID: 0000:01:00.0
2020-04-17 16:27:02.070144: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Su

## Load tf

In [16]:
def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

In [17]:
tf_graph = load_pb('../InsightFace-v2/pretrained/embedder.pb')

In [21]:
output_tensor = tf_graph.get_tensor_by_name('emb_output:0')
input_tensor = tf_graph.get_tensor_by_name('emb_input:0')

In [22]:
input_tensor, output_tensor

(<tf.Tensor 'emb_input:0' shape=(?, 3, 112, 112) dtype=float32>,
 <tf.Tensor 'emb_output:0' shape=(?, 512) dtype=float32>)

In [36]:
sess = tf.Session(graph=tf_graph)

In [ ]:
# Freeze
output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf_graph.as_graph_def(), # The graph_def is used to retrieve the nodes 
            ['emb_output'] # The output node names are used to select the usefull nodes
        )

In [7]:
op = tf_graph.get_operation_by_name('layer4.0.bn1.bias')
op.values()[0].eval(session=sess)

array([0.08987103, 0.10390206, 0.10823283, 0.13970585, 0.09509745,
       0.10193318, 0.09269255, 0.11760894, 0.07901153, 0.12126366,
       0.14068188, 0.12823322, 0.10630636, 0.1320104 , 0.08373202,
       0.10721615, 0.09942969, 0.09386835, 0.09851722, 0.10653797,
       0.12600394, 0.14321734, 0.1040848 , 0.10772728, 0.09546528,
       0.11944138, 0.0973637 , 0.11751559, 0.11859918, 0.11220834,
       0.12780198, 0.10520203, 0.08635035, 0.08463231, 0.12024499,
       0.11375844, 0.15781905, 0.12892386, 0.08233526, 0.09931961,
       0.09300137, 0.09985075, 0.10025949, 0.08362431, 0.11821734,
       0.12633796, 0.08297142, 0.09691806, 0.10430572, 0.09187757,
       0.08531639, 0.11983588, 0.06630612, 0.10769996, 0.1209331 ,
       0.10393722, 0.09300263, 0.0970154 , 0.08473245, 0.07602981,
       0.11476003, 0.10005835, 0.08136223, 0.17764395, 0.0832727 ,
       0.11189663, 0.10811879, 0.12849173, 0.08795251, 0.10371482,
       0.10448461, 0.09717975, 0.12058515, 0.12083497, 0.10510

In [63]:
np.mean([op for op in tf_graph.get_operations() if op.name == 'layer3.20.conv1.weight'][0].values()[0].eval(session=sess))
np.std([op for op in tf_graph.get_operations() if op.name == 'layer3.20.conv1.weight'][0].values()[0].eval(session=sess))

0.011334547

In [24]:
with tf_graph.as_default():
    print(tf.trainable_variables())

[]


In [78]:
dummy_input112[0, 0, 0, 1]

tensor(-0.1902, device='cuda:0')

In [23]:
output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        )

In [10]:
tf_graph.get_tensor_by_name('emb_output:0').consumers()

[]

In [31]:
output_tensor = tf_graph.get_tensor_by_name('emb_output:0')
input_tensor = tf_graph.get_tensor_by_name('emb_input:0')

In [32]:
with tf_graph.as_default():
    new_input_tensor = tf.placeholder(tf.float32, [None, 3, 112, 112])

In [33]:
output_tensor_ = ge.graph_replace(output_tensor, {input_tensor: new_input_tensor})

In [40]:
tf_graph.get_tensor_by_name('Placeholder_1:0').consumers()

[<tf.Operation 'split_1_2' type=Split>]

In [41]:
output_tensor_

<tf.Tensor 'emb_output_2:0' shape=(32, 512) dtype=float32>

In [ ]:
!ls

In [14]:
ge.graph_replace()

<function tensorflow.contrib.graph_editor.transform.graph_replace(target_ts, replacement_ts, dst_scope='', src_scope='', reuse_dst_scope=False)>

In [20]:
output_tensor.consumers()

[]

In [37]:
output = sess.run(emb_output, feed_dict={emb_input: dummy_input.cpu() * torch.ones((4, 1, 1, 1))})

NameError: name 'emb_output' is not defined

In [13]:
output

array([[-1.42374068e-01,  3.85366753e-02, -9.80976224e-02,
         2.74020601e-02, -1.81450978e-01,  1.12110779e-01,
         7.72211701e-02,  4.80393544e-02, -6.49409890e-02,
         1.14562578e-01, -1.29106596e-01,  4.83200140e-02,
         1.85366601e-01, -2.18237024e-02, -3.00251972e-02,
         8.73062164e-02,  1.69186816e-02,  5.38432077e-02,
         5.03425747e-02,  2.50295550e-02,  2.48322915e-02,
         2.54303783e-01,  4.27016392e-02,  6.64239451e-02,
         4.28727083e-02,  6.42133132e-02,  2.53222972e-01,
        -7.92215914e-02, -1.67256773e-01, -2.53816992e-02,
         4.78014722e-02,  1.38458386e-01, -8.80792215e-02,
         6.94691157e-03,  2.25715920e-01, -5.84789775e-02,
         3.31767023e-01, -8.92647579e-02,  1.31441027e-01,
        -8.04777071e-02,  9.66553614e-02, -5.64930961e-02,
         1.14102541e-02, -3.76472436e-02, -1.43959939e-01,
        -8.84618685e-02, -1.54033512e-01,  3.57127041e-02,
         4.78886180e-02,  6.54905960e-02,  1.01738989e-0

In [5]:
dummy_output

tensor([[-1.3329e-01,  5.3893e-03, -1.7111e-01, -2.4481e-03, -8.6497e-02,
          1.2130e-01,  9.2641e-02,  8.5321e-02, -1.6518e-02,  1.2433e-01,
         -1.2798e-01, -5.0267e-03,  1.2011e-01, -9.2986e-02, -1.3817e-01,
          2.0500e-01,  6.0183e-02, -1.1102e-03,  2.9920e-02, -6.9308e-03,
         -1.3100e-02,  1.2938e-01,  8.8438e-02,  1.6574e-01, -1.1849e-03,
         -3.0795e-02,  3.0578e-01, -4.5400e-02, -2.3995e-01, -9.2579e-02,
          1.2561e-01,  2.0944e-01, -1.9697e-01, -3.7204e-02,  2.6892e-01,
         -6.7064e-02,  2.5396e-01, -2.8040e-02,  1.3326e-01, -9.0823e-02,
          1.1326e-01, -2.8356e-02,  6.8087e-02, -5.6172e-03, -5.9448e-02,
         -8.8555e-02, -1.0007e-01,  1.5463e-02,  3.8660e-02,  1.4899e-01,
          1.6852e-01,  2.4980e-01, -1.2605e-01, -1.3345e-02, -1.3747e-01,
          2.8503e-01, -2.8580e-01, -3.3330e-01,  1.1535e-01,  1.4521e-01,
          5.5306e-02, -8.4521e-02, -7.4164e-02, -3.2559e-02,  2.4924e-02,
          3.2761e-01, -1.2486e-01, -1.